# Busca Competitiva


In [1]:
import numpy as np
from pprint import pprint

EMPTY = 0
PLAYER_ONE = 1
PLAYER_TWO = 2
KING_ONE = 3
KING_TWO = 4

PIECES = {
    PLAYER_ONE: (PLAYER_ONE, KING_ONE),
    KING_ONE: (PLAYER_ONE, KING_ONE),
    PLAYER_TWO: (PLAYER_TWO, KING_TWO),
    KING_TWO: (PLAYER_TWO, KING_TWO),
}

POSSIBLE_MOVES = {
    PLAYER_ONE: np.array([
        [ 1,  1],
        [ 1, -1],
    ]),
    PLAYER_TWO: np.array([
        [-1,  1],
        [-1, -1],
    ]),
    KING_ONE: np.array([
        [ 1,  1],
        [ 1, -1],
        [-1,  1],
        [-1, -1],
    ]),
    KING_TWO: np.array([
        [ 1,  1],
        [ 1, -1],
        [-1,  1],
        [-1, -1],
    ])
}


class Draughts(object):
    def __init__(self, board=None, size=6):
        self.board = np.zeros((size, size), dtype=int) if board is None else board
        self.winner = None
    
    def setup_game(self):
        board_size = len(self.board)

        # Setup initial pieces positions
        for column in range(board_size):
            if column % 2 == 0:
                self.board[0][column] = PLAYER_ONE
                # self.board[2][column] = PLAYER_ONE

                self.board[board_size - 2][column] = PLAYER_TWO
            else:
                self.board[1][column] = PLAYER_ONE

                # self.board[board_size - 2][column] = PLAYER_TWO
                self.board[board_size - 1][column] = PLAYER_TWO
    
    def get_moves(self, player):
        jumping_moves = dict()
        regular_moves = dict()

        # Gambiarra to get both player and king position
        pieces = np.where(np.isin(self.board, PIECES[player]) == True)

        board_size = len(self.board)

        for position in zip(*pieces):
            player = self.board[position]

            for move in POSSIBLE_MOVES[player]:
                position_array = np.array(position)
                line, column = position_array + move

                # Position is inside limits
                if 0 <= line < board_size and 0 <= column < board_size:
                    # Move to empty position
                    if self.board[line, column] == EMPTY:
                        regular_moves.setdefault(position, list())
                        regular_moves[position].append((line, column))
                    
                    # Player can't move if there are pieces in the position
                    elif self.board[line, column] in PIECES[player]:
                        pass

                    # There is a adversary in the position
                    else:
                        line += move[0]
                        column += move[1]
                        if 0 <= line < board_size and 0 <= column < board_size:
                            # Verify if next position is empty
                            if self.board[line, column] == EMPTY:
                                jumping_moves.setdefault(position, list())
                                jumping_moves[position].append((line, column))

        # Player must jump if possible
        return jumping_moves if len(jumping_moves) > 0 else regular_moves
    
    def play(self, start, end):
        player = self.board[start]

        # Removes player from position
        self.board[start] = EMPTY

        # Put player in next position
        self.board[end] = player
        
        # Jumping move
        distance = start[0] - end[0] if start[0] > end[0] else end[0] - start[0]
        if  distance == 2:  # Distance from start to end is 2
            # Get average position
            x = abs(start[0] + end[0]) // 2
            y = abs(start[1] + end[1]) // 2

            adversary_position = (x, y)
            # print(f"Jumping over {adversary_position}")

            self.board[adversary_position] = EMPTY
        
        # Turn players into kings
        if player == PLAYER_ONE and end[0] == len(self.board) - 1:
            # Player 1 reach the end of the board
            self.board[end] = KING_ONE
        
        if player == PLAYER_TWO and end[0] == 0:
            # Player 2 reach the start of the board
            self.board[end] = KING_TWO


    def is_over(self, moves, player):
        # There are no pieces or moves left
        if len(moves) == 0:
            # Current player lost the game
            self.winner = PLAYER_TWO if player == PLAYER_ONE else PLAYER_ONE
            return True  
        
        return False
    
    def print_board(self):
        print(end="  ")
        for index in range(len(self.board)):
            print(f"  {index}", end="")
        print()

        for index, line in enumerate(self.board):
            print(f'{index} | ', end="")
            for char in line:
                # Players
                if char < 3:
                    print(f"P{char}" if char != 0 else "  ", end=" ")
                # Kings
                else:
                    print(f"K{char // 2}" if char != 0 else "  ", end=" ")

            print("|")

draughts = Draughts()
draughts.setup_game()
draughts.print_board()

    0  1  2  3  4  5
0 | P1    P1    P1    |
1 |    P1    P1    P1 |
2 |                   |
3 |                   |
4 | P2    P2    P2    |
5 |    P2    P2    P2 |


In [2]:
import random
from copy import deepcopy
import operator


MAX_DEPTH = 4

ROUND = {
    PLAYER_ONE: max,
    PLAYER_TWO: min
}

INITIAL_SCORE = {
    PLAYER_ONE: -float("inf"),
    PLAYER_TWO:  float("inf")
}

OPERATOR = {
    PLAYER_ONE: operator.gt,
    PLAYER_TWO: operator.lt
}

def heuristic(draughts, player):
    if draughts.winner is not None:
        # If winner is defined the game is over
        return 10 if draughts.winner == player else 0

    # np.where returns 2 arrays with X and Y positions
    pieces = np.where(np.isin(draughts.board, PIECES[player]) == True)
    return len(pieces[0])

def random_move(draughts, player, adversary):
    moves = draughts.get_moves(player)
    
    position = random.choice(list(moves.keys()))
    position_moves = moves[position]

    move = random.choice(position_moves)

    random_move = (position, move)

    return random_move

def minimax(draughts, player, adversary, depth=MAX_DEPTH, heuristic=heuristic):
    moves = draughts.get_moves(player)
    
    if depth == 0 or draughts.is_over(moves, player):
        score = heuristic(draughts, player)
        return score, None

    function = ROUND[player]
    max_score = INITIAL_SCORE[player]
    operator = OPERATOR[player]
    
    for start in moves:
        positions = moves[start]

        for end in positions:
            new_draughts = deepcopy(draughts)
            new_draughts.play(start, end)

            score, _ = minimax(new_draughts, adversary, player, depth - 1)

            if operator(score, max_score):
                max_score = score
                best_move = (start, end)
        
    return max_score, best_move

In [3]:
# Adversarial search
draughts = Draughts()

# Initial state
draughts.setup_game()

player = PLAYER_ONE
adversary = PLAYER_TWO
moves = draughts.get_moves(player)
print("Starting game:")
draughts.print_board()

round = 1

# Verify if the game is over
while not draughts.is_over(moves, player):
    # Get the best move
    # position, move = random_move(draughts, player, adversary)
    socre, (position, move) = minimax(draughts, player, adversary)
    
    print(f"\nRound {round}: Player {player}")
    print(f"Moving from {position} to {move}")

    # Play the move
    draughts.play(position, move)

    draughts.print_board()

    # Set the next player
    player, adversary = adversary, player

    # Get possible moves
    moves = draughts.get_moves(player)

    round += 1

print(f"Winner: Player {draughts.winner}")

Starting game:
    0  1  2  3  4  5
0 | P1    P1    P1    |
1 |    P1    P1    P1 |
2 |                   |
3 |                   |
4 | P2    P2    P2    |
5 |    P2    P2    P2 |

Round 1: Player 1
Moving from (1, 1) to (2, 2)
    0  1  2  3  4  5
0 | P1    P1    P1    |
1 |          P1    P1 |
2 |       P1          |
3 |                   |
4 | P2    P2    P2    |
5 |    P2    P2    P2 |

Round 2: Player 2
Moving from (4, 0) to (3, 1)
    0  1  2  3  4  5
0 | P1    P1    P1    |
1 |          P1    P1 |
2 |       P1          |
3 |    P2             |
4 |       P2    P2    |
5 |    P2    P2    P2 |

Round 3: Player 1
Moving from (2, 2) to (4, 0)
    0  1  2  3  4  5
0 | P1    P1    P1    |
1 |          P1    P1 |
2 |                   |
3 |                   |
4 | P1    P2    P2    |
5 |    P2    P2    P2 |

Round 4: Player 2
Moving from (4, 2) to (3, 1)
    0  1  2  3  4  5
0 | P1    P1    P1    |
1 |          P1    P1 |
2 |                   |
3 |    P2             |
4 | P1          